## Definição da formulação forte

Considere $V$ como o espaço formado por funções $v:\overline{\Omega}\rightarrow\mathbb{R}$, com $v$ suficientemente regular e satisfazendo a condição $v(x)=0,\; \forall x\in\Gamma$.

\begin{align}
\left\{
\begin{aligned}
&-\alpha \Delta u(x) + \beta u(x) = f(x),\quad x\in\Omega,
\\[4pt]
&u(x) = 0, \quad x\in\Gamma,
\end{aligned}\right.
\end{align}

em que, $\Omega$ é um subconjunto do $\mathbb{R}^2$, $\Gamma$ é a fronteira de $\Omega$ e $\overline{\Omega}=\Omega\cup\Gamma$.


### Transição entre a formulação forte e fraca

Considere $V$ como o espaço formado por funções $v:\overline{\Omega}\rightarrow\mathbb{R}$, com $v$ suficientemente regular e satisfazendo a condição $v(x)=0,\; \forall x\in\Gamma$.

Dado $v \in V$, ao multiplicar a equação diferencial da formulação forte por $v$ e integrar sobre $\Omega$, obtemos


\begin{align*}
-\alpha\int_\Omega u_{x_1^{}x_1^{}}^{}(x)v(x)d\Omega
-\alpha\int_\Omega u_{x_2^{}x_2^{}}^{}(x)v(x)d\Omega
+\beta\int_\Omega u(x)v(x)d\Omega
= 
  \int_\Omega f(x)v(x)d\Omega .
\end{align*}

Com isso, dado que

\begin{align*}
- \int_\Omega u_{x_i^{}x_i^{}}^{}(x)v(x)d\Omega =
  \int_\Omega u_{x_i^{}}^{}(x)v_{x_i^{}}^{}(x)d\Omega,
\end{align*}


Dessa forma temos

\begin{align*}
\alpha\int_\Omega u_{x_1^{}}^{}(x)v_{x_1^{}}^{}(x)d\Omega
+\alpha\int_\Omega u_{x_2^{}}^{}(x)v_{x_2^{}}^{}(x)d\Omega
+\beta\int_\Omega u(x)v(x)d\Omega
= 
  \int_\Omega f(x)v(x)d\Omega.
\end{align*}

## Definição da formulação fraca

Dada uma função $f:\overline{\Omega}\rightarrow\mathbb{R}$ e constantes reais $\alpha>0$ e $\beta\geq 0$, determine $u\in V$ tal que

\begin{align*}
\kappa(u,v) = (f,v),\quad \forall v\in V.
\end{align*}

In [15]:
using LinearAlgebra
using Plots
using GaussQuadrature
using SparseArrays
using DataFrames

### Exemplos de solução exata para o problema acima:


In [16]:
function Exemplo1()
    alpha = 1.0 
    beta = 1.0
    gamma = 1.0
    f(x) = (alpha*π^2 + beta) * sin(π*x) + gamma * π * cos(π*x)
    u(x) = sin(π*x)
    return alpha, beta, gamma, f, u
end

function Exemplo2()
    alpha = 1.0 
    beta = 0.0
    gamma = 0.0
    f(x) = 8.0
    u(x) = -4.0 * x * (x - 1.0)
    return alpha, beta, gamma, f, u
end

"Retorna os valores de `alpha`, `beta`, `f` e `u` de acordo com o exemplo escolhido."
function dados_de_entrada(exemplo::Int)
    if exemplo == 1
        return Exemplo1()
    elseif exemplo == 2
        return Exemplo2()
    else
        error("Exemplo inválido. Escolha 1, 2, 3 ou 4.")
    end
end
     

dados_de_entrada

## Problema aproximado - Galerkin

Dada uma função $f:\overline{\Omega}\rightarrow\mathbb{R}$ e constantes reais $\alpha>0$ e $\beta\geq 0$, determine $u\in V_m$ tal que

\begin{align*}
\kappa(u,v) = (f,v),\quad \forall v\in V_m.
\end{align*}

sendo $V_m$ um subespaço vetorial de $V$ com dimensão finita $m$.

### Transição entre o problema aproximado e sua forma matriz-vetor


A formulação matricial é dada por $\displaystyle u_h(x) = \sum_{j=1}^{m} c_j\varphi_j(x)$ para $i=1,2,\dots,m$. Do sistema resultante obtemos:

\begin{align*}
\left\{
\begin{aligned}
& \kappa\big(\sum_{j=1}^mc_j\varphi_j,\varphi_1 \big) = (f,\varphi_1), 
\\
& \kappa\big(\sum_{j=1}^mc_j\varphi_j,\varphi_2 \big) = (f,\varphi_2),
\\
& \kappa\big(\sum_{j=1}^mc_j\varphi_j,\varphi_3 \big) = (f,\varphi_3),
\\
& \vdots
\\
& \kappa\big(\sum_{j=1}^mc_j\varphi_j,\varphi_m \big) = (f,\varphi_m).
\end{aligned}\right.
\end{align*}

Com isso, dado que o operador k é linear em cada componente, temos:

\begin{align*}
\left\{
\begin{aligned}
& \kappa\big(\varphi_1,\varphi_1\big)c_1 + \kappa\big(\varphi_2,\varphi_1\big)c_2 + \kappa\big(\varphi_3,\varphi_1\big)c_3 + \dots + \kappa\big(\varphi_m,\varphi_1\big)c_m = (f,\varphi_1), 
\\
& \kappa\big(\varphi_1,\varphi_2\big)c_1 + \kappa\big(\varphi_2,\varphi_2\big)c_2 + \kappa\big(\varphi_3,\varphi_2\big)c_3 + \dots + \kappa\big(\varphi_m,\varphi_2\big)c_m = (f,\varphi_2), 
\\
& \kappa\big(\varphi_1,\varphi_3\big)c_1 + \kappa\big(\varphi_2,\varphi_3\big)c_2 + \kappa\big(\varphi_3,\varphi_3\big)c_3 + \dots + \kappa\big(\varphi_m,\varphi_3\big)c_m = (f,\varphi_3), 
\\
& \vdots
\\
& \kappa\big(\varphi_1,\varphi_m\big)c_1 + \kappa\big(\varphi_2,\varphi_m\big)c_2 + \kappa\big(\varphi_3,\varphi_m\big)c_3 + \dots + \kappa\big(\varphi_m,\varphi_m\big)c_m = (f,\varphi_m).
\end{aligned}\right.
\end{align*}

Daí podemos transfomar na forma de matrizes

\begin{align*}
\begin{bmatrix}
\kappa\big(\varphi_1,\varphi_1\big)&\kappa\big(\varphi_2,\varphi_1\big)&\kappa\big(\varphi_3,\varphi_1\big)&\dots&\kappa\big(\varphi_m,\varphi_1\big)\\
\kappa\big(\varphi_1,\varphi_2\big)&\kappa\big(\varphi_2,\varphi_2\big)&\kappa\big(\varphi_3,\varphi_2\big)&\dots&\kappa\big(\varphi_m,\varphi_2\big)\\
\kappa\big(\varphi_1,\varphi_3\big)&\kappa\big(\varphi_2,\varphi_3\big)&\kappa\big(\varphi_3,\varphi_3\big)&\dots&\kappa\big(\varphi_m,\varphi_3\big)\\
\vdots&\vdots&\vdots&\ddots& \vdots  \\
\kappa\big(\varphi_1,\varphi_m\big)&\kappa\big(\varphi_2,\varphi_m\big)&\kappa\big(\varphi_3,\varphi_m\big)&\dots&\kappa\big(\varphi_m,\varphi_m\big)\\
\end{bmatrix}
\begin{bmatrix}
c_1\\
c_2\\
c_3\\
\vdots\\
c_m
\end{bmatrix}
=
\begin{bmatrix}
(f,\varphi_1)\\
(f,\varphi_2)\\
(f,\varphi_3)\\
\vdots\\
(f,\varphi_m)\\
\end{bmatrix}.
\end{align*}


## Definição da forma matriz-vetor do problema aproximado

Dada a matriz $K$ e o vetor $F$, determine o vetor $c\in\mathbb{R}^m$ tal que

\begin{align*}
Kc=F,
\end{align*}

onde

\begin{align*}
K_{i,j} = \kappa\big(\varphi_j,\varphi_i\big)
\quad\text{e}\quad
F_i = (f,\varphi_i), 
\quad\text{com}\quad
i,j\in\{1,2,\dots,m\}.
\end{align*}


In [17]:
"""
Mapeia um ponto `ξ` do intervalo padrão `[-1, 1]` para o intervalo físico
"""
function X(csi::Float64, h::Float64, p::Float64)
    return (csi + 1.0) * (h / 2.0) + p
end

function phi(csi1, csi2, a)
	if a == 1
		return (1-csi1)*(1-csi2)/4
	elseif a == 2
		return (1+csi1)*(1-csi2)/4
	elseif a == 3
		return (1+csi1)*(1+csi2)/4
	elseif a == 4
		return (1-csi1)*(1+csi2)/4
	else
		error("a deve ser 1, 2, 3 ou 4.")
	end
end

function dphi_dcsi1(csi1, csi2, a) 
	if a == 1
		return -(1-csi2)/4
	elseif a == 2
		return  (1-csi2)/4
	elseif a == 3
		return  (1+csi2)/4
	elseif a == 4
		return -(1+csi2)/4
	else
		error("a deve ser 1, 2, 3 ou 4.")
	end
end

function dphi_dcsi2(csi1, csi2, a)
	if a == 1
		return -(1-csi1)/4
	elseif a == 2
		return -(1+csi1)/4
	elseif a == 3
		return  (1+csi1)/4
	elseif a == 4
		return  (1-csi1)/4
	else
		error("a deve ser 1, 2, 3 ou 4.")
	end
end



dphi_dcsi2 (generic function with 1 method)

## Calculo das matrizes LG e EQ a partir do número de elementos


A matriz $LG$ armazena a correspondência entre a nomenclatura local e global de cada elemento finito. Assim, ela possui $n_e$ colunas, cada uma relativa a um elemento finito.

\begin{align*}
LG = 
\begin{bmatrix}
%%%%%%%%%%%%%%% Bloco 1
\begin{bmatrix}
\\
\hbox{Bloco }1\\
\\
\end{bmatrix}_{4\times N_{x_1}},
%%%%%%%%%%%%%%% Bloco 2
\begin{bmatrix}
\\
\hbox{Bloco }2\\
\\
\end{bmatrix}_{4\times N_{x_1}},
\dots,
%%%%%%%%%%%%%%% Bloco N_{x_2}
\begin{bmatrix}
\\
\hbox{Bloco }N_{x_2}\\
\\
\end{bmatrix}_{4\times N_{x_1}}
\end{bmatrix}
\end{align*}

Cada bloco da matriz $LG$ é dada por
\begin{align}
%%%%%%%%%%%%%%%%%%%%%%%% Primeiro bloco, i.e., e=1,..., N_{x_1}
\hbox{Bloco }1 = &
\begin{bmatrix}
1         & 2         & \dots &  n_{x_1}-1 \\
2         & 3         & \dots &  n_{x_1}   \\
n_{x_1}+2 & n_{x_1}+3 & \dots & 2n_{x_1}   \\
n_{x_1}+1 & n_{x_1}+2 & \dots & 2n_{x_1}-1
\end{bmatrix}_{4\times N_{x_1}},
\\[10pt]
%%%%%%%%%%%%%%%%%%%%%%%% Segundo bloco, i.e., e=N_{x_1}+1,..., 2N_{x_1}
\hbox{Bloco }2 = &
\begin{bmatrix}
 n_{x_1}+1 &  n_{x_1}+2 & \dots & 2n_{x_1}-1 \\
 n_{x_1}+2 &  n_{x_1}+3 & \dots & 2n_{x_1}   \\
2n_{x_1}+2 & 2n_{x_1}+3 & \dots & 3n_{x_1}   \\
2n_{x_1}+1 & 2n_{x_1}+2 & \dots & 3n_{x_1}-1
\end{bmatrix}_{4\times N_{x_1}},
\\[10pt]
%%%%%%%%%%%%%%%%%%%%%%%% 
\vdots
\\[10pt]
%%%%%%%%%%%%%%%%%%%%%%%% Último bloco, i.e., e=N_{x_2}-1)N_{x_1}+1,...,N_{x_2}N_{x_1}
\hbox{Bloco }N_{x_2} = &
\begin{bmatrix}
(n_{x_2}-2)n_{x_1}+1 & (n_{x_2}-2)n_{x_1}+2 & \dots & (n_{x_2}-1)n_{x_1}-1 \\
(n_{x_2}-2)n_{x_1}+2 & (n_{x_2}-2)n_{x_1}+3 & \dots & (n_{x_2}-1)n_{x_1}   \\
(n_{x_2}-1)n_{x_1}+2 & (n_{x_2}-1)n_{x_1}+3 & \dots &  n_{x_2}   n_{x_1}   \\
(n_{x_2}-1)n_{x_1}+1 & (n_{x_2}-1)n_{x_1}+2 & \dots &  n_{x_2}   n_{x_1}-1 \\
\end{bmatrix}_{4\times N_{x_1}}.
\end{align}

In [18]:
function make_partial_LG(nx1, nx2)
    return transpose(
        hcat(
            (nx2 - 2) * nx1 + 1:(nx2 - 1) * nx1 - 1, 
            (nx2 - 2) * nx1 + 2:(nx2 - 1) * nx1, 
            (nx2 - 1) * nx1 + 2:nx2 * nx1, 
            (nx2 - 1) * nx1 + 1:nx2 * nx1 - 1
        )
    )
end

function make_LG(Nx1::Int, Nx2::Int) :: Matrix{Int}
    LG = make_partial_LG(Nx1 + 1, 2)

    for i = 3:Nx2 + 1
        LG = hcat(LG, make_partial_LG(Nx1 + 1, i))
    end
    return LG
end

function make_EQ(Nx1::Int64, Nx2::Int64) :: Tuple{Int64,Vector{Int64}}
    nx1 = Nx1 + 1
    nx2 = Nx2 + 1
    m = (nx1 - 2) * (nx2 - 2)
    EQ = fill(m + 1, nx1 * nx2)
    L = []
    for j in 1:nx2-2
        for i in 0:nx1-3
            push!(L, i + (j * nx1) + 2)
        end
    end
    EQ[L] .= 1:m
    
    return m, EQ
end

make_EQ (generic function with 1 method)

### Cálculo da matriz local $K^e$ - via quadratura gaussia

\begin{align}
K_{a,b}^e 
=&
 \alpha\int_{\Omega^e} 
\frac{\partial\varphi_b^e}{\partial x_1}(x)
\frac{\partial\varphi_a^e}{\partial x_1}(x)
d\Omega
+\alpha\int_{\Omega^e} 
\frac{\partial\varphi_b^e}{\partial x_2}(x)
\frac{\partial\varphi_a^e}{\partial x_2}(x)
d\Omega
+\beta \int_{\Omega^e} 
\varphi_b^e(x)
\varphi_a^e(x)
d\Omega

\\[20pt]
= &
\alpha\int_{-1}^1\int_{-1}^1
\frac{\partial\varphi_b^e}{\partial x_1}\big(x(\xi)\big)
\frac{\partial\varphi_a^e}{\partial x_1}\big(x(\xi)\big)
\left|\frac{\partial(x_1,x_2)}{\partial(\xi_1,\xi_2)}\right| d\xi_1\,d\xi_2
\\[10pt]
& +
\alpha\int_{-1}^1\int_{-1}^1
\frac{\partial\varphi_b^e}{\partial x_2}\big(x(\xi)\big)
\frac{\partial\varphi_a^e}{\partial x_2}\big(x(\xi)\big)
\left|\frac{\partial(x_1,x_2)}{\partial(\xi_1,\xi_2)}\right| d\xi_1\,d\xi_2
\\[10pt]
& +
\beta \int_{-1}^1\int_{-1}^1
\varphi_b^e\big(x(\xi)\big)
\varphi_a^e\big(x(\xi)\big)
\left|\frac{\partial(x_1,x_2)}{\partial(\xi_1,\xi_2)}\right| d\xi_1\,d\xi_2

\\[20pt]
=&
\alpha\int_{-1}^1\int_{-1}^1
\frac{\partial\phi_b}{\partial \xi_1}(\xi)\frac{2}{h_1}
\frac{\partial\phi_a}{\partial \xi_1}(\xi)\frac{2}{h_1}
\frac{h_1h_2}{4} d\xi_1\,d\xi_2
\\[10pt]
& +
\alpha\int_{-1}^1\int_{-1}^1
\frac{\partial\phi_b}{\partial \xi_2}(\xi)\frac{2}{h_2}
\frac{\partial\phi_a}{\partial \xi_2}(\xi)\frac{2}{h_2}
\frac{h_1h_2}{4} d\xi_1\,d\xi_2
\\[10pt]
& +
\beta \int_{-1}^1\int_{-1}^1
\phi_b(\xi)
\phi_a(\xi)
\frac{h_1h_2}{4} d\xi_1\,d\xi_2

\\[20pt]
=&
\alpha
\frac{h_2}{h_1}
\int_{-1}^1\int_{-1}^1
\frac{\partial\phi_b}{\partial \xi_1}(\xi)
\frac{\partial\phi_a}{\partial \xi_1}(\xi)
d\xi_1\,d\xi_2
\\[10pt]
& +
\alpha
\frac{h_1}{h_2}
\int_{-1}^1\int_{-1}^1
\frac{\partial\phi_b}{\partial \xi_2}(\xi)
\frac{\partial\phi_a}{\partial \xi_2}(\xi)
d\xi_1\,d\xi_2
\\[10pt]
& +
\beta
\frac{h_1h_2}{4} 
\int_{-1}^1\int_{-1}^1
\phi_b(\xi)
\phi_a(\xi)
d\xi_1\,d\xi_2.
\end{align}

In [19]:

function make_Ke(alfa::Float64, beta::Float64, h1::Float64, h2::Float64, P::Vector{Float64}, W::Vector{Float64})
    K = zeros(4, 4)
    c1 = alfa * (h2 / h1)
    c2 = alfa * (h1 / h2)
    c3 = beta * (h1 * h2 / 4)

    for a in 1:4
        for b in 1:4
            for gi in 1:length(P)
                xi1 = P[gi]
                for gj in 1:length(P)
                    xi2 = P[gj]
                    K[a, b] += W[gi] * W[gj] * (
                        c1 * dphi_dcsi1(xi1, xi2, b) * dphi_dcsi1(xi1, xi2, a) + 
                        c2 * dphi_dcsi2(xi1, xi2, b) * dphi_dcsi2(xi1, xi2, a) + 
                        c3 * phi(xi1, xi2, b) * phi(xi1, xi2, a)
                    )
                end
            end
        end
    end
    return K
end



make_Ke (generic function with 1 method)

### Montagem da matriz global K a partir da matriz local $K^e$

In [20]:
function make_K(alpha::Float64, beta::Float64, n_x1::Int64, n_x2::Int64, m::Int64, EQoLG::Matrix{Int64})
    h1 = 1 / n_x1
    h2 = 1 / n_x2
    ne = n_x1 * n_x2
    P, W = legendre(2)

    ke = monta_ke(alpha, beta, h1, h2, P, W)
    K = spzeros(m+1, m+1)
    for e = 1:ne
        for b = 1:4
            j = EQoLG[b,e]
            for a = 1:4
                i = EQoLG[a,e]
                K[i,j] += ke[a,b]
            end
        end
    end
   
    return K[1:m, 1:m]
end

make_K (generic function with 1 method)

### Cálculo do vetor local $F^e$ - via quadratura gaussiana

\begin{align}
F_a^e 
=& \int_{\Omega^e}f(x)\varphi_a^e(x)d\Omega
\\[5pt]
=& \int_{-1}^1\int_{-1}^1 f\big(x(\xi)\big)\varphi_a^e\big(x(\xi)\big)
\left|\frac{\partial(x_1,x_2)}{\partial(\xi_1,\xi_2)}\right| d\xi_1\,d\xi_2
\\[5pt]
=& \frac{h_1h_2}{4}\int_{-1}^1\int_{-1}^1 f\big(x(\xi)\big) \phi_a(\xi) d\xi_1\,d\xi_2.
\end{align}

In [21]:
function make_Fe(f::Function,  h1::Float64, h2::Float64, xi1::Float64, xi2::Float64, P::Vector{Float64}, W::Vector{Float64})
    Fe = zeros(4)
    J = h1*h2/4
    for a in 1:4
        for gi in 1:length(P)
            for gj in 1:length(P)
                x1 = X(P[gi], h1, xi1)
                x2 = X(P[gj], h2, xi2)
                Fe[a] += W[gi] * W[gj] * f(x1, x2) * phi(P[gi], P[gj], a) * J
            end
        end
    end
    
    return Fe
end

make_Fe (generic function with 1 method)

### Montagem do vetor global F a partir do vetor local $F^e$


In [22]:
function make_F(f::Function, ne::Int, m::Int, EQoLG::Matrix{Int}, npg::Int=5)
    h = 1.0 / ne
    F = zeros(m + 1)
    P, W = legendre(npg)
    
    for e in 1:ne
        Fe = make_Fe(f, h, (e-1)*h, P, W, npg)
        for a = 1:2
            F[EQoLG[a, e]] += Fe[a]
        end
    end
    
    return F[1:m]
end

make_F (generic function with 2 methods)

### Cálculo do erro na norma L2 utilizando quadratura gaussiana

In [23]:
function erro_norma_L2(u::Function, c::Vector{Float64}, ne::Int, EQoLG::Matrix{Int})
    h = 1.0 / ne       
    erro = 0.0         
    P, W = legendre(5) 

    phi_1P = phi.(P, 1)
    phi_2P = phi.(P, 2)

    for e = 1:ne
        c1e = c[EQoLG[1,e]]
        c2e = c[EQoLG[2,e]]

        for j = 1:length(P)
            x = X(P[j], h, (e-1)*h) 
            erro += W[j] * (u(x) - c1e*phi_1P[j] - c2e*phi_2P[j])^2
        end
    end

    return sqrt(erro*h/2)
end

erro_norma_L2 (generic function with 1 method)

### Comparando a solução aproximada com a solução exata


In [24]:
function show_solution(case::Int)
    alfa, beta, gamma, f, u = dados_de_entrada(case)
    npg = 5
    # Define o número de elementos em que o domínio [0,1] é dividido
    ne = 4
    
    # Define a matriz LG
    LG = make_LG(ne)
    
    # Define a dimensão do espaço aproximado Vₘ e o vetor EQ
    m, EQ = make_EQ(ne)
    
    # Define a matriz EQoLG
    EQoLG = EQ[LG]
    
    # Exibe os valores ne, LG, m, EQ e EQoLG
    display("ne = "); display(ne)
    display("LG = "); display(LG)
    display("m = "); display(m)
    display("EQ = "); display(EQ)
    display("EQoLG = "); display(EQoLG)
    
    
    # Constrói a matriz K e o vetor F
    K = make_K(alfa, beta, gamma, ne, m, EQoLG)
    F = make_F(f, ne, m, EQoLG, npg)
    
    # Exibe a matriz K (sistema linear) e o vetor F (vetor força)
    display("Matriz K:")
    display(K)
    display("Vetor F:")
    display(F)
    
    # Resolve o sistema linear K * c = F para obter os coeficientes solução aproximada u_h(x)
    c = K \ F
    display("Solução aproximada U:")
    display(c)
    
    # Exibe a solução exata nos nos internos da malha
    display("Solução exata nos nós internos:")
    h = 1.0 / ne  # Comprimento de cada elemento finito
    display(u.(h:h:1-h))
    
    # Define a discretização da malha com N nós internos, incluindo os pontos de fronteira
    malha = 0:h:1
    
    # Gera o gráfico comparando a solução aproximada com a solução exata
    plt1 = plot(0:0.01:1, u.(0:0.01:1), label="Exata", lw=3, title="Comparação: Solução Exata vs. Aproximada")
    plot!(plt1, malha, [0; c; 0], label="Aproximada", lw=3, linestyle=:solid, markershape=:circle) # O "!" adiciona ao gráfico existente
    xlabel!("x")  # Adiciona o rótulo ao eixo x
    
    # Exibe o gráfico final
    display(plt1) 
end

show_solution (generic function with 1 method)

### Estudo de convergência do erro.

In [25]:
function show_error(case::Int)
    alfa, beta, gamma, f, u = dados_de_entrada(case)

    npg = 5

    P, W = legendre(npg)

    errors = zeros(9)
    vec_h = zeros(9)

    for p in 2:10
        ne = 2^p - 1
        h = 1.0 / ne
        vec_h[p-1] = h

        LG = make_LG(ne)
        m, EQ = make_EQ(ne)
        EQoLG = EQ[LG]

        K = make_K(alfa, beta, gamma, ne, m, EQoLG)
        F = make_F(f, ne, m, EQoLG, npg)

        c = K \ F

        errors[p-1] = erro_norma_L2(u, [c;0], ne, EQoLG)
    end


    # Gera o gráfico comparando a solução aproximada com a solução exata
    plt2 = Plots.plot(vec_h, errors, yscale=:log10, xscale=:log10, label="Errors", xlabel="h", ylabel="Erro", title="Erros na solução do problema")
    plot!(plt2, vec_h, vec_h.^2, yscale=:log10, xscale=:log10, legend=:topleft, label="h^2", xlabel="h", ylabel="Erro")
    xlabel!("x")  # Adiciona o rótulo ao eixo x

    # Exibe o gráfico final
    display(plt2)
end

show_error (generic function with 1 method)

In [26]:
case = 1
show_solution(case)

MethodError: MethodError: no method matching make_LG(::Int64)
Closest candidates are:
  make_LG(::Int64, !Matched::Int64) at ~/Workspace/metodos-numericos/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:12

In [27]:
case = 1
show_error(case)

MethodError: MethodError: no method matching make_LG(::Int64)
Closest candidates are:
  make_LG(::Int64, !Matched::Int64) at ~/Workspace/metodos-numericos/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:12